In [1]:
# 📍 1. Imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from pathlib import Path

In [2]:
# 📍 2. Load & Preprocess
df = pd.read_csv("../data/processed/labeled_messages.csv")
df["date_parsed"] = pd.to_datetime(df["date_parsed"], errors="coerce")
df = df.dropna(subset=["date_parsed"])

# Sentiment to numeric
sentiment_map = {"Positive": 1, "Negative": -1, "Neutral": 0}
df["Sentiment_Score"] = df["Sentiment"].map(sentiment_map)

# Message features
df["char_count"] = df["text"].astype(str).apply(len)
df["word_count"] = df["text"].astype(str).apply(lambda x: len(x.split()))
df["Month"] = df["date_parsed"].dt.to_period("M")

In [3]:
# 📍 3. Group & Feature Engineering
monthly_df = df.groupby(["from", "Month"]).agg({
    "text": "count",
    "char_count": "mean",
    "word_count": "mean",
    "Sentiment_Score": "sum"
}).reset_index()

monthly_df = monthly_df.rename(columns={
    "from": "Employee",
    "text": "msg_count",
    "char_count": "avg_msg_length",
    "word_count": "avg_word_count",
    "Sentiment_Score": "sentiment_score"
})


In [4]:
# 📍 4. Train/Test Split
features = ["msg_count", "avg_msg_length", "avg_word_count"]
X = monthly_df[features]
y = monthly_df["sentiment_score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# 📍 5. Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# ------------------------------------------------------------
# 📍 6. Evaluation
print("R^2 Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# Coefficients
coef_df = pd.DataFrame({
    "Feature": features,
    "Coefficient": model.coef_
})
print(coef_df)

R^2 Score: 0.42765020804350784
RMSE: 1.994649338696962
          Feature  Coefficient
0       msg_count     0.347254
1  avg_msg_length    -0.004865
2  avg_word_count     0.039418
